<a href="https://colab.research.google.com/github/FrankQixiangGao/Algorithms-Coursera/blob/main/HW7_QXG190000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import files
import io
import os
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

uploaded = files.upload()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Saving federalist.csv to federalist (1).csv


In [30]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded["federalist.csv"]))
df.head()

,author,text
0,HAMILTON,FEDERALIST. No. 1 General Introduction For the...
1,JAY,FEDERALIST No. 2 Concerning Dangers from Forei...
2,JAY,FEDERALIST No. 3 The Same Subject Continued (C...
3,JAY,FEDERALIST No. 4 The Same Subject Continued (C...
4,JAY,FEDERALIST No. 5 The Same Subject Continued (C...


In [31]:
#Step 1, Convert the author column to categorical data
authors = set(df['author'])
authors = {author :idx for author, idx in zip(authors, range(len(authors)))}
df['author'] = df['author'].map(lambda x: authors[x])
df.head()

,author,text
0,3,FEDERALIST. No. 1 General Introduction For the...
1,2,FEDERALIST No. 2 Concerning Dangers from Forei...
2,2,FEDERALIST No. 3 The Same Subject Continued (C...
3,2,FEDERALIST No. 4 The Same Subject Continued (C...
4,2,FEDERALIST No. 5 The Same Subject Continued (C...


In [32]:
df['author'].value_counts()

3    49
4    15
0    11
2     5
1     3
Name: author, dtype: int64

In [23]:
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['author'], test_size = 0.2, random_state = 1234)

In [33]:
print('training text shape', train_texts.shape)
print('validatation text shape', val_texts.shape)
print("training label shape", train_labels.shape)
print("validatation label shape", val_labels.shape)

training text shape (66,)
validatation text shape (17,)
training label shape (66,)
validatation label shape (17,)


In [36]:
#Text Processing step 3
#remove stopword and performing tf-idf vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = stopwords.words('english'))
vectorizer.fit(train_texts)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [37]:
train_A = vectorizer.transform(train_texts)
val_A = vectorizer.transform(val_texts)

In [38]:
#Step 4, Try a Bernoulli Naive Bayes Model, What is your accuracy on the test set?
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(train_A, train_labels)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [41]:
print("Training accuracy", nb.score(train_A, train_labels))
print("Training accuracy", nb.score(val_A, val_labels))

Training accuracy 0.7727272727272727
Training accuracy 0.5882352941176471


In [42]:
#redo the vectorization with max_features option set to use only the 1000 most frequent words

vectorizer = TfidfVectorizer(stop_words= stopwords.words('english'), max_features=1000, ngram_range=(1,2))
vectorizer.fit(train_texts)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=1000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [43]:
train_A = vectorizer.transform(train_texts)
val_A = vectorizer.transform(val_texts)

In [45]:
# retrain the model with only 1000 max-features
nb = BernoulliNB()
nb.fit(train_A, train_labels)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [46]:
#Print the Training Accuracy
print("Training accuracy", nb.score(train_A, train_labels))
print("Training accuracy", nb.score(val_A, val_labels))

Training accuracy 1.0
Training accuracy 0.9411764705882353


In [53]:
#Using Logistic Regression
from sklearn.linear_model import LogisticRegression
class_count = df['author'].value_counts()
class_count /= sum(class_count)
class_weight = {label : weight for label, weight in 
                zip(list(class_count.index), class_count)}

In [59]:
log_reg = LogisticRegression(class_weight = class_weight)
log_reg.fit(train_A, train_labels)
print("Training accuracy", log_reg.score(train_A, train_labels))
print("Validation accuracy", log_reg.score(val_A, val_labels))

Training accuracy 0.5909090909090909
Validation accuracy 0.5882352941176471


In [57]:
#  no parameter logisticRegression
log_reg = LogisticRegression()
log_reg.fit(train_A, train_labels)
print("Training accuracy", log_reg.score(train_A, train_labels))
print("Validation accuracy", log_reg.score(val_A, val_labels))

Training accuracy 0.7121212121212122
Validation accuracy 0.5882352941176471


In [60]:
#Based on this experiement, I can improve LogisticRegression() result by having no parameters. And Compared to Naiive Bayes, the Logistic Regression doesn't have a high performance

In [70]:
from sklearn.neural_network import MLPClassifier
neural = MLPClassifier(hidden_layer_sizes = (150, 50),
                   activation = 'relu', solver = 'lbfgs',
                    random_state = 1000, verbose = False,
                    validation_fraction = 0.15,
                     alpha = 10e-4,
                   tol = 10e-5, max_iter = 300)
neural.fit(train_A, train_labels)
print("Training accuracy", neural.score(train_A, train_labels))
print("Validation accuracy", neural.score(val_A, val_labels))

Training accuracy 1.0
Validation accuracy 0.8235294117647058


In [ ]:
#MLP Classifier after all perform very well, close to Naiive Bayes. It is Validation accurcy is around 82%